In [7]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')
print(sys.path)

['/home/hieutt/RD-OpenMax_CAN_IDS/notebooks', '/home/hieutt/miniconda3/envs/torchtf/lib/python39.zip', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [3]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from tfrecord.writer import TFRecordWriter  # NOT tensorflow's writer!
import os
import torch
from tfrecord.torch.dataset import TFRecordDataset
from tqdm import tqdm

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1]:
import tfrecord
print(tfrecord.__file__)

/home/hieutt/miniconda3/envs/torchtf/lib/python3.9/site-packages/tfrecord/__init__.py


In [9]:
# 📄 Step 2: Define TFRecord schema
description = {
    'id_seq': 'int',
    'data_seq': 'int',
    'timestamp': 'float',
    'delta_ts': 'float',
    'label': 'int'
}

In [10]:
# 🔧 Step 3: Define helper functions

def parse_sample(data, target_size=64):
    id_seq = torch.tensor(data['id_seq'], dtype=torch.float32).view(target_size, target_size)
    data_seq = torch.tensor(data['data_seq'], dtype=torch.float32).view(target_size, target_size)
    timestamp = torch.tensor(data['timestamp'], dtype=torch.float32).view(target_size, target_size)
    delta_ts = torch.tensor(data['delta_ts'], dtype=torch.float32).view(target_size, target_size)
    label = int(data['label'][0])
    return (id_seq, data_seq, timestamp, delta_ts, label)

def serialize_example(x):
    id_seq, data_seq, timestamp, delta_ts, label = x
    features = tf.train.Features(feature={
        'id_seq': tf.train.Feature(float_list=tf.train.FloatList(value=id_seq.view(-1).tolist())),
        'data_seq': tf.train.Feature(float_list=tf.train.FloatList(value=data_seq.view(-1).tolist())),
        'timestamp': tf.train.Feature(float_list=tf.train.FloatList(value=timestamp.view(-1).tolist())),
        'delta_ts': tf.train.Feature(float_list=tf.train.FloatList(value=delta_ts.view(-1).tolist())),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label])),
    })
    return tf.train.Example(features=features).SerializeToString()




In [11]:
def build_oe_tfrecord(input_dirs, output_file, target_size=64):
    index_path = output_file + ".index"
    writer = TFRecordWriter(output_file, index_path=index_path)
    total_written = 0

    all_tfrec_files = []
    for input_dir in input_dirs:
        tfrec_files = sorted([
            os.path.join(input_dir, f)
            for f in os.listdir(input_dir)
            if f.endswith(".tfrec")
        ])
        all_tfrec_files.extend(tfrec_files)

    print(f"[INFO] Found {len(all_tfrec_files)} TFRecord files in total.")

    for fname in tqdm(all_tfrec_files, desc="Processing TFRecords"):
        dataset = TFRecordDataset(fname, index_path=None, description=description)
        for data in dataset:
            id_seq, data_seq, timestamp, delta_ts, label = parse_sample(data, target_size)
            if label != 0:
                serialized = serialize_example((id_seq, data_seq, timestamp, delta_ts, -1))
                writer.write(serialized)
                total_written += 1

    writer.close()
    print(f"\n✅ DONE: {total_written} OE samples written to: {output_file}")
    print(f"📄 Index file saved to: {index_path}")

In [12]:
input_dirs = [
    "../data/set_04/test_03_known_vehicle_unknown_attack/preprocessed/one_type/dos/TFRecord_w64_s32/2/val",
    "../data/set_04/test_03_known_vehicle_unknown_attack/preprocessed/one_type/force-neutral/TFRecord_w64_s32/2/val",
]

output_file = "../data/set_04/test_03_known_vehicle_unknown_attack/preprocessed/oe_record/oe.tfrecord"

build_oe_tfrecord(input_dirs, output_file)


TypeError: __init__() got an unexpected keyword argument 'index_path'